In [34]:
import os
import csv
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [35]:
# helper functions

def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]


def unroll(series):
    l = []
    for key in series:
        if (key[-1] == 'v' or key[-1] == 'z'): continue
        l += (series[key])
    return l

def E3(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= (7.0/18.0)):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E2(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= (5.0/18.0)):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E1(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= (3.0/18.0)):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E0(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= (1.0/18.0)):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def eval(pred, truth):
    o_pred = []
    o_truth = []
    f_pred = []
    f_truth = []
    b_pred = []
    b_truth = []
    s_pred = []
    s_truth = []
    l_pred = []
    l_truth = []
    for p in pred:
        o_pred.append(p[0])
        f_pred.append(p[1])
        b_pred.append(p[2])
        s_pred.append(p[3])
        l_pred.append(p[4])
    for t in truth:
        o_truth.append(t[0])
        f_truth.append(t[1])
        b_truth.append(t[2])
        s_truth.append(t[3])
        l_truth.append(t[4])
    return [[E0(o_pred, o_truth), E1(o_pred, o_truth), E2(o_pred, o_truth), E3(o_pred, o_truth)],
            [E0(f_pred, f_truth), E1(f_pred, f_truth), E2(f_pred, f_truth), E3(f_pred, f_truth)],
            [E0(b_pred, b_truth), E1(b_pred, b_truth), E2(b_pred, b_truth), E3(b_pred, b_truth)],
            [E0(s_pred, s_truth), E1(s_pred, s_truth), E2(s_pred, s_truth), E3(s_pred, s_truth)],
            [E0(l_pred, l_truth), E1(l_pred, l_truth), E2(l_pred, l_truth), E3(l_pred, l_truth)]]

In [36]:
# loading data

raw_X_train_list = []
smoothed_X_train_list = []
trans_X_train_list = []
final_X_train_list = []

y_train_list = []

raw_X_test_list = []
smoothed_X_test_list = []
trans_X_test_list = []
final_X_test_list = []

y_test_list = []

with open("..\\test_examples.txt") as t:
    test_examples = t.readlines()

for example in test_examples:
    raw_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\1_unprocessed\\" + example[:-1])
    smoothed_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\2_smoothed\\" + example[:-1])
    trans_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\3_translation\\" + example[:-1])
    final_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\4_final\\" + example[:-1])

    raw_X_test_list.append(unroll(raw_series))
    smoothed_X_test_list.append(unroll(smoothed_series))
    trans_X_test_list.append(unroll(trans_series))
    final_X_test_list.append(unroll(final_series))
    
    y_test_list.append([float(example[4]) / 9, float(example[7]) / 9, float(example[10]) / 9, float(example[13]) / 9, float(example[16]) / 9])

raw_X_test = torch.tensor(raw_X_test_list)
smoothed_X_test = torch.tensor(smoothed_X_test_list)
trans_X_test = torch.tensor(trans_X_test_list)
final_X_test = torch.tensor(final_X_test_list)


y_test = torch.tensor(y_test_list)

print(raw_X_test.size())
print(smoothed_X_test.size())
print(trans_X_test.size())
print(final_X_test.size())
print(y_test.size())
print('\n\n')

with open("..\\training_examples.txt") as t:
    training_examples = t.readlines()

for example in training_examples:
    raw_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\1_unprocessed\\" + example[:-1])
    smoothed_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\2_smoothed\\" + example[:-1])
    trans_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\3_translation\\" + example[:-1])
    final_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\4_final\\" + example[:-1])

    raw_X_train_list.append(unroll(raw_series))
    smoothed_X_train_list.append(unroll(smoothed_series))
    trans_X_train_list.append(unroll(trans_series))
    final_X_train_list.append(unroll(final_series))
    
    y_train_list.append([float(example[4]) / 9, float(example[7]) / 9, float(example[10]) / 9, float(example[13]) / 9, float(example[16]) / 9])

raw_X_train = torch.tensor(raw_X_train_list)
smoothed_X_train = torch.tensor(smoothed_X_train_list)
trans_X_train = torch.tensor(trans_X_train_list)
final_X_train = torch.tensor(final_X_train_list)

y_train = torch.tensor(y_train_list)

print(raw_X_train.size())
print(smoothed_X_train.size())
print(trans_X_train.size())
print(final_X_train.size())
print(y_train.size())

torch.Size([93, 224])
torch.Size([93, 224])
torch.Size([93, 224])
torch.Size([93, 224])
torch.Size([93, 5])



torch.Size([371, 224])
torch.Size([371, 224])
torch.Size([371, 224])
torch.Size([371, 224])
torch.Size([371, 5])


In [37]:
# setting up FCNN

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)

input_size = 224
hidden_layer_size_1 = 450
hidden_layer_size_2 = 400
hidden_layer_size_3 = 330
hidden_layer_size_4 = 250
hidden_layer_size_5 = 150
hidden_layer_size_6 = 50
output_size = 5
batch_size = 371
learning_rate = 0.001

model = nn.Sequential(nn.Linear(input_size, hidden_layer_size_1),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_1, hidden_layer_size_2),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_2, hidden_layer_size_3),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_3, hidden_layer_size_4),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_4, hidden_layer_size_5),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_5, hidden_layer_size_6),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_6, output_size),
                        nn.Sigmoid())

loss_function = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

training_cycles = 20
num_epochs = 100

In [38]:
# training and evaluating raw series

min_loss = 1.0

errors = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(raw_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(raw_X_test)
        test_loss = loss_function(test_y, y_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (test_losses [-1] < min_loss):
        min_loss = test_losses[-1]
        test_y = model(raw_X_test)

        errors.clear()
        errors = eval(test_y, y_test.tolist()) 

print('MSE: ' + str(min_loss))
print('ERRORS: ' + str(errors))

MSE: 0.05724222585558891
ERRORS: [[17.204301075268816, 55.913978494623656, 75.26881720430107, 83.87096774193549], [13.978494623655914, 64.51612903225806, 84.94623655913979, 91.39784946236558], [12.903225806451612, 36.55913978494624, 61.29032258064516, 89.24731182795699], [15.053763440860216, 52.68817204301075, 83.87096774193549, 93.54838709677419], [12.903225806451612, 60.215053763440864, 80.64516129032258, 89.24731182795699]]


In [39]:
# training and evaluating smoothed series

min_loss = 1.0

errors = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(smoothed_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(smoothed_X_test)
        test_loss = loss_function(test_y, y_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (test_losses [-1] < min_loss):
        min_loss = test_losses[-1]
        test_y = model(smoothed_X_test)

        errors.clear()
        errors = eval(test_y, y_test.tolist())

print('MSE: ' + str(min_loss))
print('ERRORS: ' + str(errors))

MSE: 0.05717817321419716
ERRORS: [[20.43010752688172, 60.215053763440864, 76.34408602150538, 84.94623655913979], [24.731182795698924, 69.89247311827957, 83.87096774193549, 89.24731182795699], [9.67741935483871, 36.55913978494624, 62.365591397849464, 88.17204301075269], [13.978494623655914, 53.763440860215056, 83.87096774193549, 94.6236559139785], [9.67741935483871, 62.365591397849464, 81.72043010752688, 89.24731182795699]]


In [40]:
# training and evaluating trans series

min_loss = 1.0

errors = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(trans_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(trans_X_test)
        test_loss = loss_function(test_y, y_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (test_losses [-1] < min_loss):
        min_loss = test_losses[-1]
        test_y = model(trans_X_test)

        errors.clear()
        errors = eval(test_y, y_test.tolist())

print('MSE: ' + str(min_loss))
print('ERRORS: ' + str(errors))

MSE: 0.051994625478982925
ERRORS: [[18.27956989247312, 54.83870967741935, 74.19354838709677, 82.79569892473118], [31.182795698924732, 67.74193548387096, 83.87096774193549, 92.47311827956989], [11.827956989247312, 37.634408602150536, 61.29032258064516, 87.09677419354838], [36.55913978494624, 65.59139784946237, 93.54838709677419, 95.6989247311828], [26.881720430107528, 72.04301075268818, 88.17204301075269, 93.54838709677419]]


In [41]:
# training and evaluating final series

min_loss = 1.0

errors = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(final_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(final_X_test)
        test_loss = loss_function(test_y, y_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (test_losses [-1] < min_loss):
        min_loss = test_losses[-1]
        test_y = model(final_X_test)

        errors.clear()
        errors = eval(test_y, y_test.tolist())

print('MSE: ' + str(min_loss))
print('ERRORS: ' + str(errors))

MSE: 0.03345710411667824
ERRORS: [[24.731182795698924, 59.13978494623656, 80.64516129032258, 90.3225806451613], [52.68817204301075, 78.49462365591398, 93.54838709677419, 98.9247311827957], [26.881720430107528, 65.59139784946237, 81.72043010752688, 90.3225806451613], [24.731182795698924, 70.96774193548387, 86.02150537634408, 98.9247311827957], [34.40860215053763, 77.41935483870968, 90.3225806451613, 94.6236559139785]]
